In [20]:
# disable compiler warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# imports 
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers import Dense
from typing import List
import datetime

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [21]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
(train_ds_global, test_ds_global), ds_info = tfds.load("mnist" , split =["train", "test"] , as_supervised=True , with_info=True)

def prepare_mnist_data(mnist):
  #convert data from uint8 to float32
  mnist = mnist.map(lambda img, target: (tf.cast(img, tf.float32), target))
  #sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
  mnist = mnist.map(lambda img, target: ((img/128.)-1., target))
  #create one-hot targets
  mnist = mnist.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
  #cache this progress in memory, as there is no need to redo it; it is deterministic after all
  mnist = mnist.cache()
  #shuffle, batch, prefetch
  mnist = mnist.shuffle(1000)
  mnist = mnist.batch(32)
  mnist = mnist.prefetch(20)
  #return preprocessed dataset
  return mnist

In [22]:
class BasicConv(tf.keras.Model):
    def __init__(self):
        super(BasicConv, self).__init__()

        self.loss_function = tf.keras.losses.CategoricalCrossentropy()
        self.metrics_list = [tf.keras.metrics.Accuracy(name="Accuracy"),
                             tf.keras.metrics.Mean(name="loss")]

        self.optimizer = tf.keras.optimizers.Adam()

        self.convlayer1 = tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu')
        self.convlayer2 = tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu')
        self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

        self.convlayer3 = tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu')
        self.convlayer4 = tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu')
        self.global_pool = tf.keras.layers.GlobalAvgPool2D()

        self.out = tf.keras.layers.Dense(10, activation='softmax')

    @tf.function
    def call(self, x):
        x = self.convlayer1(x)
        x = self.convlayer2(x)
        x = self.pooling(x)
        x = self.convlayer3(x)
        x = self.convlayer4(x)
        x = self.global_pool(x)
        x = self.out(x)
        return x

    # 3. metrics property
    @property
    def metrics(self):
        return self.metrics_list
        # return a list with all metrics in the model



    # 4. reset all metrics objects
    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()



    # train_step method
    @tf.function
    def train_step(self, data):
        img, label = data
        
        # compute output and loss, train the variables
        with tf.GradientTape() as tape:
            output = self(img, training=True)
            loss = self.loss_function(label, output)
            
        # update trainable variables
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # update metrics
        self.metrics_list[0].update_state(tf.argmax(output, axis=1), tf.argmax(label, axis=1))
        self.metrics_list[1].update_state(loss)
        
        # return a dict with metric information
        return {m.name : m.result() for m in self.metrics_list}



    # test_step method
    @tf.function
    def test_step(self, data):
        img, label = data

        # compute output and loss, without training
        output = self(img, training=False)
        loss = self.loss_function(label, output)

        # update metrics
        self.metrics_list[0].update_state(tf.argmax(output, axis=1), tf.argmax(label, axis=1))
        self.metrics_list[1].update_state(loss)

        # return a dict with metric information 
        return {m.name : m.result() for m in self.metrics_list}

In [23]:

def create_summary_writers(config_name):
    
    # Define where to save the logs
    # along with this, you may want to save a config file with the same name so you know what the hyperparameters were used
    # alternatively make a copy of the code that is used for later reference
    
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    train_log_path = f"logs/{config_name}/{current_time}/train"
    val_log_path = f"logs/{config_name}/{current_time}/val"

    # log writer for training metrics
    train_summary_writer = tf.summary.create_file_writer(train_log_path)

    # log writer for validation metrics
    val_summary_writer = tf.summary.create_file_writer(val_log_path)
    
    return train_summary_writer, val_summary_writer

train_summary_writer, val_summary_writer = create_summary_writers(config_name="RUN1")

In [24]:
# trains the model by iterating through the dataset and applying training_step method epochs time
def training_loop(model, train_ds, epochs, train_summary_writer):
    # iterate over epochs
    for epoch in range(epochs):
        print(f"epoch: {epoch}")

        # train steps on all batches in the training data
        for (img, label) in train_ds:
            metrics = model.train_step((img, label))
            
            # keep data in summary with metrics
            with train_summary_writer.as_default():
                for metric in model.metrics_list:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)
        
        # print current metric values and reset the metrics
        print([f"{key} : {value.numpy()}" for (key, value ) in metrics.items()])
        model.reset_metrics()


# tests overall performance of model
def test_loop(model, test_ds, val_summary_writer):
    # test steps on every item in test dataset
    for (img, label) in test_ds:
        metrics = model.test_step((img, label))
        
        # keep data with metrics
        with val_summary_writer.as_default():
            for metric in model.metrics_list:
                tf.summary.scalar(f"{metric.name}", metric.result(), step=1)
    
    #print current metric values and reset the metrics
    print([f"{key} : {value.numpy()}" for (key, value ) in metrics.items()])
    model.reset_metrics()

In [25]:
"""def try_model(model, ds):
  for x, t in ds.take(5):
    y = model(x)
    print(tf.argmax(y, axis=1), tf.argmax(t, axis=1))

basic_model = BasicConv()
try_model(basic_model, train_dataset)"""

train_dataset = train_ds_global.apply(prepare_mnist_data)
test_dataset = test_ds_global.apply(prepare_mnist_data)

print(test_dataset)

basic_model = BasicConv()

# about 10% accuracy, makes sense, since output is always the same and there are 10 labels
test_loop(basic_model,
        test_ds=test_dataset,
        val_summary_writer=val_summary_writer)

training_loop(basic_model,
              train_ds=train_dataset,
              epochs=10, 
              train_summary_writer=train_summary_writer)

test_loop(basic_model,
        test_ds=test_dataset,
        val_summary_writer=val_summary_writer)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>
['Accuracy : 0.0982000008225441', 'loss : 2.30375599861145']
epoch: 0
['Accuracy : 0.817716658115387', 'loss : 0.5548437237739563']
epoch: 1
['Accuracy : 0.9539499878883362', 'loss : 0.15196023881435394']
epoch: 2
['Accuracy : 0.9676666855812073', 'loss : 0.10751833021640778']
['Accuracy : 0.9660999774932861', 'loss : 0.11859608441591263']


In [28]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6007 (pid 11429), started 0:00:39 ago. (Use '!kill 11429' to kill it.)